![](images/GH.jpg)


# <center><ins> Life Expectancy and Healthcare </ins></center>

## Overview

**Global Health** is always a public topic at the center. Many aspects are related to **Global Health**. Among them, **Life Expectancy** is a very important measure. WHO has published the complete **2020 World Health Statistics**. My project will focus on the **Life Expectancy** and some selected attributes within the **Healthcare category**, such as **Healthcare Coverage**, **Healthcare Workforce**. I will try to find out possible links between **Life Expectancy** and **Healthcare**.

<br><br>

## The Data


### Data Source:

[https://www.kaggle.com/utkarshxy/who-worldhealth-statistics-2020-complete](https://www.kaggle.com/utkarshxy/who-worldhealth-statistics-2020-complete)


### About the Data:

This dataset covers the most recent and updated global health statistics by WHO. It is a set of 39 files, each corresponding to a specific aspect of the health statistics. For my analysis purpose, I choose from them 8 datasets relating to **Life Expectancy** and **Healthcare**:

* Life expectancy at birth, country wise by years
* Universal healthcare coverage
* Percentage of population with household expenditures on healthcare greater than 10% of total household income
* Percentage of population with household expenditures on healthcare greater than 25% of total household income
* Medical doctors per 10,000 population.
* Nurses per 10,000 population.
* Dentists per 10,000 population
* Pharmacists per 10,000 population

A brief exploration showed that the yearly statistics are not available for many of the years. All these datasets have only **2015** in common. **2015** is not far from now, so all data for this year are selected for further analysis.



### First glance at the data:

![](images/le.png)


This picture shows the top and bottom 20 Countries on Life Expectancy. Unfortunately USA is not among the top 20.

![](images/boxplot.png)

This figure is a boxplot for all the features I selected. In order to put all the values in more or less the same scale, I normalized each feature by dividing the values in each column by the mean of that column. This figure looks strange. That is because there is an outlier for the Nurses column, its value is more than 30 times the average. I checked，it is for the country Belize, there are 1744 nurses in 10,000 population, while other countries have around 100. This is an obvious outlier probably due to data recording mistake. This boxplot is really a handy tool to find out if you have any outliers in your data.

### How badly could a single outlier affect the result?

![](images/bad.jpg)

Look at the correlation coefficients calculated with Pandas corr() method. The result for Nurses shows that Life Expectancy is poorly correlated to nurse numbers, in contrast to being strongly correlated to doctor and pharmacist numbers. This sounds unreasonable. Later we will see the real correlation after removing this single outlier.

### Second glance at the data:

![](images/boxplot_new.png)

After removing this single outlier, now the boxplot looks reasonable. It shows that Life Expectancy and Universal Healthcare Coverage have narrow data range, others are more diversified. Some of them have several outliers, but not so extreme. I will keep them.

### Third glance at the data:

![](images/good.jpg)

Now the correlations are similar for doctor, nurse, dentist and pharmacist numbers. The following two figures are seaborn correlation heatmap and pandas scatter matrix plot.

![](images/heatmap.png)

![](images/scatter_matrix.png)


## Hypothesis Testing


### Shapiro-Wilk Normality Test:

The above correlation matrix is calculated using the Pandas default Pearson's method which assumes the variables are normally distributed. To validate this calculation, it is necessary to test if the various features in the dataframe are normally distributed.

* **H0: the sample is normally distributed.**
* **H1: the sample is not normally distributed.**
* **Threshold $\alpha$: 0.05**

Codes for the tesing:

```Python
def test_normality(data):
    for col in data.columns[1:]:
        stat, p = stats.shapiro(data[col].dropna())
        print('For {0}, stat={1:.3f}, p={2:.3f}'.format(col, stat, p))
        if p > 0.05:
            print(f'{col} is probably normally distributed.\n')
        else:
            print(f'{col} is probably not normally distributed.\n')
            
test_normality(df)
```
</center>

![](images/normality.jpg)

The p-values are all smaller than the threshold, we should reject the null hypothesis, so none of the samples is normally distributed.


### Spearman Correlation Tests:

The Spearman's rank correlation coefficient test is often used to establish whether two variables may be regarded as statistically dependent. Unlike the Pearson's Correlation test which assumes the random variables are normally distributed, the Spearman's test does not rely on any assumptions on the distributions of the variables, namely this test is **non-parametric**.

Spearman's test is based on the following assumptions:

* **Observations in each sample are independent and distributed identically.**
* **Observations in each sample are ranked.**

The hypotheses are:

* **H0: the two samples do not have any correlation.**
* **H1: the two samples are correlated.**
* **Threshold $\alpha$: 0.05**

Since we don't know if the samples are positively correlated or negativelly correlated, so it is a two-sided test.

The codes for the testing is simple:
```Python
corr_, p_value = stats.spearmanr(df.iloc[:,1:], nan_policy='omit')
corr_ = corr_.round(decimals=3)
p_value = p_value.round(decimals=3)

```


**Here is the summary table showing the correlation between features: I listed only those related to Life Expectancy**

Feature Pair | Correlation Coefficient | p-Value
------------ | ------------- | -------------
LifeExpectancy / uhcCoverage | 0.883 | 0.0
LifeExpectancy / Pharmacists | 0.757 | 0.0
LifeExpectancy / Nurses | 0.732 | 0.0
LifeExpectancy / Doctors | 0.722 | 0.0
LifeExpectancy / Dentists | 0.672 | 0.0
LifeExpectancy / HHExpdt10 | 0.302 | 0.161
LifeExpectancy / HHExpdt25 | 0.085 | 0.701

You can see the Life Expectancy is most correlated to the Universal Healthcare Coverage, which is not surprising. It also strongly correlated to Healthcare Workforce, like doctor, nurse and pharmacist numbers. It's interesting to see that it is correlated to dentist numbers to a lesser degree. On the contrary, it is not correlated to percentage of household income spent on the healthcare, which I think is explainable. The percentage of household income is just a percentage, it's not reflecting the total amount of money spent on healthcare. Should we have the data of total cost, we might be able to find some correlation.


### The 95% confidence intervals by formula:

**Spearman Test is testing on non-parametric distributions, which means we don't have parameters for this kind of distribution, so we also don't have pdf() or ppf() to calculate the confidence interval.** 

I found from the following resource the formula to calculate the 95% confidence interval:
<br>

### <B><center>$\tanh(\operatorname{arctanh}r\pm1.96/\sqrt{n-3})$</center></B>


Reference: 
[https://stats.stackexchange.com/questions/18887/how-to-calculate-a-confidence-interval-for-spearmans-rank-correlation](https://stats.stackexchange.com/questions/18887/how-to-calculate-a-confidence-interval-for-spearmans-rank-correlation)



Here is the code for calculation:

```Python
# make a list of the labels of all relevent columns
col = df.columns[1:]

# define a function to take in the correlation coefficients array from the Spearman's Test, 
# and return a dictionary of the confidence intervals for each coefficient
def spearman_ci(data):
    dict_ = {}
    for i in range(len(data)):
        for j in range(i+1,len(data)):
            num = len(df[[col[i],col[j]]].dropna())
            stderr = 1.0 / math.sqrt(num - 3)
            delta = 1.96 * stderr
            r = data[i][j]
            lower = round(math.tanh(math.atanh(r) - delta), 3)
            upper = round(math.tanh(math.atanh(r) + delta), 3)
            dict_[f'{col[i]} / {col[j]}'] = (lower,upper)
    return dict_
    
ci = spearman_ci(corr_)
```

<br>

**This is the updated table with confidence interval: looks reasonable**

Feature Pair | Correlation Coefficient | p-Value | Confidence Interval
------------ | ------------- | ------------- | -------------
LifeExpectancy / uhcCoverage | 0.883 | 0.0 | (0.846, 0.911)
LifeExpectancy / Pharmacists | 0.757 | 0.0 | (0.644, 0.838)
LifeExpectancy / Nurses | 0.732 | 0.0 | (0.639, 0.804)
LifeExpectancy / Doctors | 0.722 | 0.0 | (0.614, 0.804)
LifeExpectancy / Dentists | 0.672 | 0.0 | (0.535, 0.775)
LifeExpectancy / HHExpdt10 | 0.302 | 0.161 | (-0.126, 0.635)
LifeExpectancy / HHExpdt25 | 0.085 | 0.701 | (-0.339, 0.48)


### The 95% confidence intervals by bootstrapping:

**Skylar English: "The Bootstrap is a tool to quantify the variation in a statistical estimate. It can be used in almost any situation."**

Inspired by Skylar's words, I wrote some codes, using the bootstrap sampling technique to calculate the confidence intervals for my test:

```Python
df_le_uhc = df[['LifeExpectancy','uhcCoverage']]

corr_test, p_test = stats.spearmanr(df_le_uhc.iloc[:, 0], df_le_uhc.iloc[:, 1])

np.random.seed(1)
bootstrap = []
for i in range(10000):
    # Have to bootstrap pairwisely, so I randomly sample the indices of the dataframe, 
    # which will in turn select the dataframe by indices
    boot = df_le_uhc.iloc[np.random.choice(df_le_uhc.shape[0], size=df_le_uhc.shape[0], replace=True)]
    corr_le_uhc, p_value = stats.spearmanr(boot)
    bootstrap.append(corr_le_uhc)
    
left = np.percentile(bootstrap, 2.5)
right = np.percentile(bootstrap, 97.5)

print("The correlation is: {:.3f}".format(stats.spearmanr(df_le_uhc.iloc[:, 0], df_le_uhc.iloc[:, 1])[0]))
print(f"Bootstrap Confidence Interval for Correlation: {left:.3f}, {right:.3f}")

fig, ax = plt.subplots(1, figsize=(12, 5))
ax.hist(bootstrap, bins=50, density=True, alpha=0.5)
ax.axvline(stats.spearmanr(df_le_uhc.iloc[:, 0], df_le_uhc.iloc[:, 1])[0], color='r')
ax.axvline(left, color='g')
ax.axvline(right, color='g')
ax.set_title("Confidence Interval by Bootstraping", fontsize=20)
ax.tick_params(axis='both', which='major', labelsize=15)
plt.tight_layout()
plt.savefig('images/bootstrap.png')

```

Here is a figure showing the simulation and confidence interval of correlation between Life Expectancy and Universal Healthcare Coverage :

![](images/bootstrap.png)

**Update the table again with the bootstrapping confidence intervals, we can see the confidence intervals calculated with two different methods are comparable.**

Feature Pair | Correlation Coefficient | p_value | Confidence Interval | Bootstrapping CI
------------ | ------------- | ------------- | ------------- | -------------
LifeExpectancy / uhcCoverage | 0.883 | 0.0 | (0.846, 0.911) | (0.838, 0.914)
LifeExpectancy / Pharmacists | 0.757 | 0.0 | (0.644, 0.838) | (0.640, 0.835)
LifeExpectancy / Nurses | 0.732 | 0.0 | (0.639, 0.804) | (0.627, 0.812)
LifeExpectancy / Doctors | 0.722 | 0.0 | (0.614, 0.804) | (0.581, 0.824)
LifeExpectancy / Dentists | 0.672 | 0.0 | (0.535, 0.775) | (0.515, 0.786)
LifeExpectancy / HHExpdt10 | 0.302 | 0.161 | (-0.126, 0.635) | (-0.164, 0.718)
LifeExpectancy / HHExpdt25 | 0.085 | 0.701 | (-0.339, 0.48) | (-0.405, 0.543)


### Some more illustrative figures:


**This is the correlation heatmap calculated using Spearman's method.**
![](images/heatmap_update.png)


**Below is the errorbar plot, showing the correlation and its confidence interval.**
![](images/corr_ci.png)

It's generally accepted that, if the correlation coefficient is greater than 0.7, the two samples are strongly correlated. In this figure, the red hexagons to the right of the green line represent strong correlations. You can also see, generally speaking, the higher the correlation, the narrower the confidence interval is. And of course, the confidence interval is also depending on the sample size.


### Fitting the data points of the most correlated feature pairs, Life Expectancy and Universal Healthcare Coverage:


![](images/scatter_fit.png)

The scatter plot with the fitted line, shows a clear positive correlation. At this point, I am not sure if it is the best fit using the first degree polynomial model. But I am pretty sure we will learn more about this in the coming weeks.

## Future plan

**From the Spearman Correlation Testing, we can see Life Expectancy is mostly correlated to Universal Healthcare Coverage, and to Healthcare Workforce at a lesser degree. A question is: Is it really that the correlation between Life Expectancy and Universal Healthcare Coverage stronger than the others? I may try out the AB testing.**

## Conclusion

### The Life Expectancy is strongly correlated to the Universal Healthcare Coverage and Healthcare Workforce. It is not correlated to the percentage of household income spent on healthcare.